# Buscar dados do Spotify

Esse notebook tem o intuito de buscar músicas do Spotify através de playlists de usuários para montar um dataset de recomendações.

### Bibliotecas

- spotipy
- pandas

### Instalação e importação de dependências

In [ ]:
pip install spotipy python-dotenv


In [5]:
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
import pandas as pd
import spotipy
import os

load_dotenv()

In [6]:
client_id = os.getenv("SPOTIFY_CLIENT_ID")
client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


Podemos fazer essa busca de duas formas:

1. Buscar as playlists com base em categorias do Spotify
2. Buscar as playlists de um usuário previamente autenticado

### Buscando com base nas categorias

In [7]:
categories = []  # Inserir as categorias
playlists = []

for category in categories:
    offset = 0
    limit = 20

    while True:
        page_results = sp.category_playlists(
            category, offset=offset, limit=limit)
        playlists += page_results["playlists"]["items"]

        if not page_results["playlists"]["next"]:
            break

        offset += limit


### Buscando com base no usuário

In [8]:
playlists = []
offset = 0
limit = 50

while True:
    page_results = sp.current_user_playlists(offset=offset, limit=limit)
    playlists += page_results["items"]

    if not page_results["next"]:
        break

    offset += limit


Agora que recebemos as playlists, podemos buscar as músicas delas.

### Criar uma lista com os IDs das playlists

In [9]:
playlists_ids = []
for playlist in playlists:
    if playlist is not None:
        playlists_ids.append(playlist["id"])

playlists_ids


[]

### Buscar as músicas das playlists

In [13]:
fields = "items(track(name, artists(name, id, popularity), id, popularity)), next"
results = []

for playlist_id in playlists_ids:
    offset = 0
    limit = 50

    while True:
        page_results = sp.playlist_tracks(
            playlist_id, fields=fields, offset=offset, limit=limit)
        results += page_results["items"]

        if not page_results["next"]:
            break

        offset += limit

print(len(results))


50


[{'track': {'artists': [{'id': '0uGWkZRPp1Urk77XBrPBsZ', 'name': 'AgroPlay'},
    {'id': '2CKOmarVWvWqkNWUatHCex', 'name': 'Ana Castela'}],
   'id': '4uqJelb9THHmJ3OCohg4ZJ',
   'name': 'Nosso Quadro',
   'popularity': 82}},
 {'track': {'artists': [{'id': '1yR65psqiazQpeM79CcGh8',
     'name': 'Marília Mendonça'}],
   'id': '2K9kZpwD2CzTa6iiSYYOoO',
   'name': 'Leão',
   'popularity': 89}},
 {'track': {'artists': [{'id': '2eK9gcJQ6uqVvJL63dnOM3',
     'name': 'Simone Mendes'}],
   'id': '51pxwIxDVCbFHW8oyJFQXD',
   'name': 'Erro Gostoso - Ao Vivo',
   'popularity': 87}},
 {'track': {'artists': [{'id': '41QLxRXlc2NwfJZkHGHKid',
     'name': 'Israel & Rodolffo'},
    {'id': '0BHm7qbh3ENxvXzkQAG7MP', 'name': 'Mari Fernandez'}],
   'id': '3PH1nUysW7ybo3Yu8sqlPN',
   'name': 'Seu Brilho Sumiu - Ao Vivo',
   'popularity': 83}},
 {'track': {'artists': [{'id': '487N2T9nIPEHrlTZLL3SQs',
     'name': 'Zé Neto & Cristiano'}],
   'id': '3mzbab4H0iZYkHggSEj0UX',
   'name': 'Oi Balde - Ao Vivo',
   

In [14]:
tracks = []

for item in results:
    if item["track"] is not None:
        track = {
            "artist_name": str(item["track"]["artists"][0]["name"]),
            "artist_url": item["track"]["artists"][0]["id"],
            "track_name": item["track"]["name"],
            "track_url": item["track"]["id"],
            "track_popularity": item["track"]["popularity"]
        }
        tracks.append(track)
    else:
        print("Skipping item with missing track")

len(tracks)


[{'artist_name': 'AgroPlay',
  'artist_url': '0uGWkZRPp1Urk77XBrPBsZ',
  'track_name': 'Nosso Quadro',
  'track_url': '4uqJelb9THHmJ3OCohg4ZJ',
  'track_popularity': 82},
 {'artist_name': 'Marília Mendonça',
  'artist_url': '1yR65psqiazQpeM79CcGh8',
  'track_name': 'Leão',
  'track_url': '2K9kZpwD2CzTa6iiSYYOoO',
  'track_popularity': 89},
 {'artist_name': 'Simone Mendes',
  'artist_url': '2eK9gcJQ6uqVvJL63dnOM3',
  'track_name': 'Erro Gostoso - Ao Vivo',
  'track_url': '51pxwIxDVCbFHW8oyJFQXD',
  'track_popularity': 87},
 {'artist_name': 'Israel & Rodolffo',
  'artist_url': '41QLxRXlc2NwfJZkHGHKid',
  'track_name': 'Seu Brilho Sumiu - Ao Vivo',
  'track_url': '3PH1nUysW7ybo3Yu8sqlPN',
  'track_popularity': 83},
 {'artist_name': 'Zé Neto & Cristiano',
  'artist_url': '487N2T9nIPEHrlTZLL3SQs',
  'track_name': 'Oi Balde - Ao Vivo',
  'track_url': '3mzbab4H0iZYkHggSEj0UX',
  'track_popularity': 87},
 {'artist_name': 'KayBlack',
  'artist_url': '2h5Ha0ZiMFmOQD3iYcSXsy',
  'track_name': 'Me

In [15]:
df = pd.DataFrame(tracks)
df.head(10)


,artist_name,artist_url,track_name,track_url,track_popularity
0,AgroPlay,0uGWkZRPp1Urk77XBrPBsZ,Nosso Quadro,4uqJelb9THHmJ3OCohg4ZJ,82
1,Marília Mendonça,1yR65psqiazQpeM79CcGh8,Leão,2K9kZpwD2CzTa6iiSYYOoO,89
2,Simone Mendes,2eK9gcJQ6uqVvJL63dnOM3,Erro Gostoso - Ao Vivo,51pxwIxDVCbFHW8oyJFQXD,87
3,Israel & Rodolffo,41QLxRXlc2NwfJZkHGHKid,Seu Brilho Sumiu - Ao Vivo,3PH1nUysW7ybo3Yu8sqlPN,83
4,Zé Neto & Cristiano,487N2T9nIPEHrlTZLL3SQs,Oi Balde - Ao Vivo,3mzbab4H0iZYkHggSEj0UX,87
5,KayBlack,2h5Ha0ZiMFmOQD3iYcSXsy,Melhor Só,2LTlykcdu0gwb7zcKneo2a,82
6,Matheus & Kauan,2Z0lRIqr997lIUiPtrpKCr,Não Vitalício (Nunca Mais) - Ao Vivo,4sqpSDnTrbS8v7RMNCUyqS,86
7,Henrique & Juliano,3p7PcrEHaaKLJnPUGOtRlT,Traumatizei - Ao Vivo Em Brasília,0JIYoXtJfORxI6gRpHv6ZW,87
8,Israel & Rodolffo,41QLxRXlc2NwfJZkHGHKid,Bombonzinho - Ao Vivo,0SCMVUZ21uYYB8cc0ScfbV,78
9,Guilherme & Benuto,6m6e7D2TnV0aYMllFFwMxu,Duas Três,0redQHBfejmPUhonR5ppGC,86


Agora com as informações das músicas, vamos salvar em um `.csv` para fazermos o pré-processamento e o processamento.

In [19]:
# Para um novo to_process.csv
df.to_csv("to_process.csv", index=False)
# Para mesclar em um to_process.csv já existente
# df.to_csv("to_process.csv", mode="a", index=False, header=False)
